In [1]:
import pandas as pd
import numpy as np
import konlpy
from konlpy.tag import Twitter
import nltk
from gensim import models
import re
import requests
import pickle

In [ ]:
with open('labeled_data.pkl', 'rb') as data:
    data_refd = pickle.load(data)
data_refd.shape

In [ ]:
data_rmd = data_refd.copy()

In [ ]:
lists = []
for i in range(42):
    if sum(data_refd[i + 1]) < 200:
        del data_rmd[i + 1]
data_rmd

In [ ]:
##################
#전체 데이터에서 콜롬 1-42 원핫코딩중 아무것도 1에 해당하지 않는 데이터제거
#남은 데이터의 갯수는 8066
remove = []
for index, row in data_rmd.iterrows():
    x = row[5:]
    if sum(x) == 0:
        remove.append(index)
len(remove)
remove[1]
for i in remove:
    data_rmd.drop(i, inplace = True)

In [ ]:
with open('reduced_data.pkl', 'wb') as data:
    pickle.dump(data_rmd, data, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
with open('reduced_data.pkl', 'rb') as data:
    data_rmd = pickle.load(data)
data_rmd.shape

(10168, 24)

In [3]:
### 띄어쓰기, 문장구분 전처리
data_cont = pd.DataFrame()
data_cont["내용"]=data_rmd.판례내용
contents = []
for index, row in data_cont.iterrows():
    r = row["내용"].replace('\r', '').replace('\n', '').replace(",", " ").replace("  ","")
    contents.append(r)
len(contents)

10168

In [4]:
######## 내용안에서 [이유] 다음 문장을 다 뽑음
def regex_filter(contents):
    order_list = []
    reason_list = []
    o =  re.compile("【주문】|【주    문】")
    p = re.compile("【이유】|【이    유】")
#     q = re.compile("재판장|대법관")
    for sen in contents:
        index = [(l.end(), m.end()) for l, m in zip(o.finditer(sen), p.finditer(sen))]
        if len(index) >= 1 :
            for i in index:
                l, m = i
                order_list.append(sen[l:m-4])
                reason_list.append(sen[m:])
        else:
                order_list.append("None")
                reason_list.append("None")
    return order_list, reason_list

order, reason = regex_filter(contents)

In [5]:
#################
### 문서에서 punctuation 제거 숫자는 의미가 잇을거 같기도해서 일단 남김
contents3 =[]
for sen in reason:
    p = re.compile("\W")
    r = re.sub(p," ", sen)
    p = re.compile("\d")
    r = re.sub(p," ", r)
    contents3.append(r)


In [15]:
twitter = Twitter()
## 위에 정리된 content3 문장에서 조사 제외, 형법제외 그리고 단어길이가 1인거 제외
## 단어길이가 1인것중에서 쓸데없고 의미없는 것들이 명사로 잡히는경우가 너무 많음...
temp = []
# for i in range(len(contents3)):a
for i in range(len(contents3)):
    ss = twitter.pos(contents3[i], norm=True, stem= True)
    ah = []
    for wh in ss:
        if wh[1] != "Josa" and wh[0] != "형법" and len(wh[0]) > 1 and wh not in common_stop_words:
            ah.append(wh)
    temp.append(ah)
len(temp)
#temp 문장을 다 뽑으니 전체에 대해서는 내 노트북에서 너무 오래걸림.. 성원이의 슈퍼컴으로 확인부탁;;

10168

In [18]:
temp[0]

[('요지', 'Noun'),
 ('서울', 'Noun'),
 ('광진구', 'Noun'),
 ('자양', 'Noun'),
 ('소재', 'Noun'),
 ('머릿돌', 'Noun'),
 ('교회', 'Noun'),
 ('신축', 'Noun'),
 ('공사', 'Noun'),
 ('현장', 'Noun'),
 ('총괄', 'Noun'),
 ('책임자', 'Noun'),
 ('공사', 'Noun'),
 ('현장', 'Noun'),
 ('주차장', 'Noun'),
 ('정화조', 'Noun'),
 ('맨홀', 'Noun'),
 ('직경', 'Noun'),
 ('cm', 'Alpha'),
 ('깊이', 'Noun'),
 ('cm', 'Alpha'),
 ('입구', 'Noun'),
 ('높이다', 'Verb'),
 ('공사', 'Noun'),
 ('작업', 'Noun'),
 ('인부', 'Noun'),
 ('오랑', 'Noun'),
 ('지시', 'Noun'),
 ('오랑', 'Noun'),
 ('장소', 'Noun'),
 ('발생', 'Noun'),
 ('안전', 'Noun'),
 ('사고', 'Noun'),
 ('예방', 'Noun'),
 ('맨홀', 'Noun'),
 ('주위', 'Noun'),
 ('펜스', 'Noun'),
 ('설치', 'Noun'),
 ('외부', 'Noun'),
 ('출입', 'Noun'),
 ('통제', 'Noun'),
 ('뚜껑', 'Noun'),
 ('맨홀', 'Noun'),
 ('구멍', 'Noun'),
 ('견고', 'Noun'),
 ('덮다', 'Verb'),
 ('놓다', 'Verb'),
 ('이러하다', 'Adjective'),
 ('조치', 'Noun'),
 ('취하다', 'Verb'),
 ('얇다', 'Adjective'),
 ('합판', 'Noun'),
 ('덮다', 'Verb'),
 ('확인', 'Noun'),
 ('점검', 'Noun'),
 ('시정', 'Noun'),
 ('지시', 'Noun'),
 ('공사', 'N

In [17]:
data_orders = {i:j for i,j in zip(data_rmd.index, order) if len(j) >0}
len(data_orders)

TypeError: object of type 'int' has no len()

In [19]:
data_reasons = {i:j for i,j in zip(data_rmd.index, temp) if len(j) >0}
len(data_reasons)

10168

In [20]:
with open('data_order.pkl', 'wb') as data:
    pickle.dump(data_orders, data, protocol=pickle.HIGHEST_PROTOCOL)
with open('data_reason.pkl', 'wb') as data:
    pickle.dump(data_reasons, data, protocol=pickle.HIGHEST_PROTOCOL)


In [21]:
with open('data_order.pkl', 'rb') as data:
    order = pickle.load(data)
with open('data_reason.pkl', 'rb') as data:
    reason = pickle.load(data)

In [22]:
token = [t for d in temp for t in d]
print(len(token))

text = nltk.Text(token, name="NMSC")
print(text.vocab().most_common(100))

6428955
[(('가다', 'Verb'), 19838), (('따르다', 'Verb'), 19539), (('업무', 'Noun'), 19363), (('회사', 'Noun'), 19297), (('처벌', 'Noun'), 19278), (('법리', 'Noun'), 18602), (('관련', 'Noun'), 18059), (('수사', 'Noun'), 17901), (('비추다', 'Verb'), 17752), (('대출', 'Noun'), 17612), (('로부터', 'Noun'), 17582), (('규정', 'Noun'), 17542), (('당시', 'Noun'), 17389), (('사업', 'Noun'), 17372), (('취지', 'Noun'), 17169), (('지급', 'Noun'), 17015), (('자금', 'Noun'), 16902), (('이다', 'Verb'), 16883), (('명의', 'Noun'), 16447), (('다음', 'Noun'), 16401), (('사용', 'Noun'), 16144), (('사정', 'Noun'), 15428), (('관계', 'Noun'), 14805), (('주식', 'Noun'), 14179), (('오해', 'Noun'), 14117), (('자신', 'Noun'), 13637), (('유죄', 'Noun'), 13515), (('로서', 'Noun'), 13463), (('이상', 'Noun'), 12805), (('계약', 'Noun'), 12721), (('재판', 'Noun'), 12670), (('모두', 'Noun'), 12641), (('방법', 'Noun'), 12541), (('목적', 'Noun'), 12488), (('의사', 'Noun'), 12480), (('이익', 'Noun'), 12368), (('공사', 'Noun'), 12211), (('종합', 'Noun'), 12167), (('법정', 'Noun'), 12108), (('정당', 'Noun

In [23]:
common_stop_words = [i[0] for i in text.vocab().most_common(100) if i[1]>20000]
common_stop_words

[]

In [25]:
reason_list = []
for i in reason.values():
    reasons =[]
    for j in i:
        if j[0] not in common_stop_words:
            reasons.append(j[0])
    reason_list.append(reasons)
len(reason_list)

10168

### (3)nltk로 wordcloud 그리기
(twitter로 명사 필터링 했습니다. stop words에 대해선 별도의 필터링 하지 않았어요. 차후에 더 수정할게요!)

In [ ]:
from matplotlib import pylab, font_manager, rc
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import platform

font_fname = "gulim.ttc" #굴림 폰트 파일 넣어놨어요
font_name = font_manager.FontProperties(fname=font_fname).get_name()
rc("font",family = font_name)

t = Twitter()

# cloud_noun = [j for i in contents3[1:1000] for j in t.nouns(i)]
cloud_noun_all = [j for i in contents3 for j in t.nouns(i)]

In [ ]:
cloud_noun_all

In [ ]:
ko = nltk.Text(cloud_noun_all, name = "워드클라우드_명사")
data = ko.vocab().most_common(500)
dict(data)

In [ ]:
ko = nltk.Text(cloud_noun_all, name = "워드클라우드_명사")
# ko = nltk.Text(cloud_noun, name = "워드클라우드_명사_1:1000")
# ko.vocab().most_common(500)
data = ko.vocab().most_common(500)

temp_data = dict(data)
wordcloud = WordCloud(font_path = font_fname,
                     relative_scaling = 0.2,
                     background_color = "white")
# WordCloud(width=900,height=500, max_words=1628,relative_scaling=1,normalize_plurals=False).generate_from_frequencies(d)
wrdcld = wordcloud.generate_from_frequencies(temp_data)

plt.figure(figsize=(16,8))
plt.imshow(wrdcld)
plt.axis=("off")
plt.show()

###  square는 너무 밋밋해서 해보는 마스킹

In [ ]:
import numpy as np
from PIL import Image
law_coloring = np.array(Image.open("./law.jpg")) #jpg파일 넣어놨습니다.

from wordcloud import ImageColorGenerator
image_colors = ImageColorGenerator(law_coloring)

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(law_coloring, cmap=plt.cm.gray, interpolation="bilinear")

In [ ]:
plt.show()

In [ ]:
wordcloud2 = WordCloud(relative_scaling=0.2,
                      mask = law_coloring,
                      background_color= "white",
                      min_font_size =1,
                      max_font_size=20).generate_from_frequencies(temp_data)

In [ ]:
plt.figure(figsize=(12,12))
plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear")
plt.show()

### 는 마스킹해도 별로 안 예쁜듯...ㅠㅠ? 다른 이미지를 구해서 해보는게 좋을듯!

#### wordcloud끝!

In [ ]:
twitter = Twitter()
## 위에 정리된 content3 문장에서 조사 제외, 형법제외 그리고 단어길이가 1인거 제외
## 단어길이가 1인것중에서 쓸데없고 의미없는 것들이 명사로 잡히는경우가 너무 많음...
temp = []
# for i in range(len(contents3)):a
for i in range(len(contents3[0:1000])):
    ss = twitter.pos(contents3[i], norm=True, stem= True)
    ah = []
    for wh in ss:
        if wh[1] != "Josa" and wh[0] != "형법" and len(wh[0]) > 1:
            ah.append(wh)
    temp.append(ah)
    
#temp 문장을 다 뽑으니 전체에 대해서는 내 노트북에서 너무 오래걸림.. 성원이의 슈퍼컴으로 확인부탁;;

In [ ]:
token = [t for d in temp for t in d]
print(len(token))

text = nltk.Text(token, name="NMSC")
print(text.vocab().most_common(100))

### (1) Tree 구조 그려보기 (는 저희가 갖고 있는 데이터가  완전한 문장이 아니고  토픽 워드 모델링 하는 용이라 원하는 POS만 걸러낸 상태라 트리 구조 그리는 게 의미 없긴 하지만 그냥 해봤어요..)

In [ ]:
test_tag = token[1:10]
parser_ko = nltk.RegexpParser("NP: {<Adjective>*<Noun>*}")
chunks_ko = parser_ko.parse(test_tag)
chunks_ko.draw()

### (2) plot부분 (혹시 필요할지 몰라) 따로 저장하기. 폰트는 굴림 체를 다운 받아서 사용했어요

In [ ]:
from matplotlib import pylab, font_manager, rc

font_fname = "c:/Users/ji/Downloads/gulim.ttc"
font_name = font_manager.FontProperties(fname=font_fname).get_name()

rc("font",family = font_name)

pylab.show = lambda: pylab.savefig("most_word.png")
text.plot(50)

# 여기서부터 doc2vec

In [ ]:
sentences = list()
cnt = 0
for i in temp:
    sentences.append(models.doc2vec.LabeledSentence(words=i, tags=["SENT_{}".format(cnt)]))
    cnt+=1

In [ ]:
len(sentences)

In [ ]:
sentences[1:10]

In [ ]:
class LabeledLineSentence(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for uid, line in enumerate(open(filename)):
            yield LabeledSentence(words=line.split(), labels=['SENT_%s' % uid])
            
model = models.Doc2Vec(alpha=.025, min_alpha=.025, min_count=1)
model.build_vocab(sentences)

for epoch in range(10):
    model.train(sentences, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= 0.002
    model.min_alpha = model.alpha
    
model.save("my_model.doc2vec")
model_loaded = models.Doc2Vec.load('my_model.doc2vec')
print(model_loaded.docvecs.most_similar(["SENT_2"]))

In [ ]:
print(model_loaded.docvecs.most_similar(["SENT_2"]))
print(temp[0])
list(data_refd.iloc[[0], 3])
## 첫문장만 토큰화됨!
## 군 형법 제외!
## 조사, 숫자, Punctuation은 빼는걸로

In [ ]:
# ref_data = data.dropna(subset=['참조조문'])
# ref_data.shape

# ref_crime_data = ref_data[ref_data.apply(lambda row: '형법' in row[3], axis = 1)]
# data[data.apply(lambda row: row['참조조문'].find('형법').any(), axis=1)]
# df.apply(lambda row: row.astype(str).str.contains('TEST').any(), axis=1)
# ref_crime_data